### MADE IT TO CHUNK 6, PROCESS 1-6!!!!

In [107]:
import pandas as pd
import numpy as np

from ipywidgets import IntProgress


In [201]:
chunkMapDF = pd.read_csv('data/ChunkMap.csv')
qrVals = pd.read_csv('QR_Values.csv')

In [192]:
def kParameters(X, Y, QA, QB):
    random_state = np.random.RandomState(0)
    F = np.eye(2) #Transition Matrix (Identity Matrix)
    Z = Y #Observation matrix
    X0 = [[4.], [.0023]] #Inital parameters guess (?) <<<< NEED VALUES
    P0 = (np.eye(2) + random_state.randn(2, 2)*.1)*100*np.eye(2) #Covariance Matrix (The confidence in our prediction)
    R = 1
    Q = [[QA,0.],[0.,QB]]
    w = np.random.multivariate_normal([0., 0.], Q)
    return {'random_state':random_state, 'F':F, 'Z':Z, 'X0':X0, 'P0':P0, 'R':R, 'Q':Q, 'w':w}

def kalmanFilter(X, Y, parameters, extended=True):
    Xs = []
    
    Pk = parameters['P0'] 
    Xk = parameters['X0'] 
    for n in range(0, len(X)):
        time, actual = X[n], Y[n]
        Pk = Pk + parameters["Q"]
                  
        Hk, Zk = 0, 0
        if(extended):
            part1 = np.exp(Xk[1][0]*time)
            Hk = [part1, part1*(Xk[0][0]*time)]
            Zk = Xk[0][0]*np.exp(Xk[1][0]*time)
        else:
            Hk = [1., time]
            Zk = np.dot(Hk, Xk) + np.random.normal(loc=0., scale=parameters['R']) #<----- Consider putting Vk instead of R        
        
        Yk = actual - Zk # residuals
        transposedH = [[Hk[0]],[Hk[1]]]
        Sk = np.dot(np.dot(Hk, Pk), transposedH)[0]+parameters['R']
        Kk = np.dot(Pk, transposedH)*(1/Sk)
        Xnext = Xk + Kk*Yk        
        p_part = (np.eye(2)-np.outer(Kk, Hk))
        Pnext = np.dot(p_part, Pk)
        Xs.append([Xk[0][0], Xk[1][0]])
        #Set new vars based on current observation
        Pk = Pnext
        Xk = Xnext
    if(len(Xs)==0):
        if(len(X) <240):
            return ['TooShort', 'TooShort']
        else:
            return [np.nan,np.nan]
    return Xs[-1:][0]

In [206]:
chunkNames = chunkMapDF['Chunk'].unique()[:]

chunkBar = IntProgress(min=0, max=len(chunkNames))
contestBar = IntProgress(min=0, max=len(contests))

print("Chunks:")
display(chunkBar)

print("Contests:")
display(contestBar)

results = []

for chunkName in chunkNames:
    print(chunkName)
    chunkDF = pd.read_csv('data/Chunks/'+chunkName+'.csv').drop(columns=['Unnamed: 0']).sort_values(by=['MinutesRemaining'], ascending=False)
    
    contests = chunkDF['ContestId'].unique()[:]
    contestBar.value=0
    contestBar.max = len(contests)
    for cid in contests:
        cSeriesDF = chunkDF[chunkDF['ContestId']==cid]
        numberToRemove = len(cSeriesDF[cSeriesDF['MinutesRemaining']<240])

        X = cSeriesDF['MinutesRemaining'].max() - cSeriesDF['MinutesRemaining'][:-numberToRemove]
        Y = cSeriesDF['Entries'].cumsum()[:-numberToRemove]
        X, Y = list(X), list(Y)        
        count = 0
        contest = {'ContestId':cid}
        for ID, qrObj in qrVals.iterrows():
            label = qrObj['label']
            P = kParameters(X, Y, float(qrObj['Q1']), float(qrObj['Q2']))
            val = kalmanFilter(X, Y, P)  
            
            contest['A'+label] = val[0]
            contest['B'+label] = val[1]
        results.append(contest)
        contestBar.value+=1
    chunkBar.value+=1


Chunks:


IntProgress(value=0, max=65)

Contests:


IntProgress(value=0, max=10000)

chunk1


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in multiply
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in double_scalars


chunk2


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in double_scalars


chunk3
chunk4
chunk5
chunk6
chunk7
chunk8
chunk9
chunk10
chunk11
chunk12
chunk13
chunk14
chunk15
chunk16
chunk17
chunk18
chunk19
chunk20
chunk21
chunk22
chunk23
chunk24
chunk25
chunk26
chunk27
chunk28
chunk29
chunk30
chunk31
chunk32
chunk33
chunk34
chunk35
chunk36
chunk37
chunk38
chunk39
chunk40
chunk41
chunk42
chunk43
chunk44
chunk45
chunk46
chunk47
chunk48
chunk49
chunk50
chunk51
chunk52
chunk53
chunk54
chunk55
chunk56
chunk57
chunk58
chunk59
chunk60
chunk61
chunk62
chunk63
chunk64
chunk65


In [207]:
rDF = pd.DataFrame(results)

In [209]:
len(rDF.dropna())

638723

In [210]:
rDF.to_csv('data/AB_Values.csv')